In [1]:
import os
from ultralytics import YOLO
from tqdm import tqdm

In [2]:
# Path
yaml_path = "dataset.yaml"  
results_dir = "results"     
os.makedirs(results_dir, exist_ok=True)

# Model
models = {
    "YOLOv3": "yolov3.pt",
    "YOLOv4": "yolov4.pt",
    "YOLOv5x": "yolov5x.pt",
    "YOLOv6": "yolov6.pt",
    "YOLOv7": "yolov7.pt",
}


In [ ]:

# Metrics store
metrics = {}

# TEC Metrics for Each Model
for model_name, model_path in tqdm(models.items(), desc="Processing Models"):
    print(f"\nTraining and evaluating {model_name}...")
    
    # Train--model
    model = YOLO(model_path)  # Load
    model.train(
        data=yaml_path,
        epochs=50,
        imgsz=640,
        batch=16,
        name=f"{model_name}_detection",
        save_dir=os.path.join(results_dir, model_name)
    )
    
    # Validate
    val_results = model.val()
    
    # Metrics
    metrics[model_name] = {
        "AP50": val_results["metrics/mAP50"],  # Average Precision at IoU=0.50
        "AP": val_results["metrics/mAP50-95"], # Average Precision (IoU=0.50:0.95)
        "Precision": val_results["metrics/precision"],
        "Recall": val_results["metrics/recall"],
        "F1-Score": val_results["metrics/F1"],
        "IoU": val_results["metrics/mIoU"],  # Mean IoU
        "Loss": val_results["metrics/loss"],
        "Speed": val_results["metrics/speed"],
    }
    
    # Save 
    model.plot_results(save_dir=os.path.join(results_dir, model_name))



In [ ]:
# Display
print("\nFinal Metrics:")
for model_name, model_metrics in metrics.items():
    print(f"\nModel: {model_name}")
    for metric, value in model_metrics.items():
        print(f"  {metric}: {value:.4f}")